In [ ]:
import sys

sys.path.append("..")

import os
from pathlib import Path

import pycolmap
import numpy as np

import matplotlib.pyplot as plt

from hloc.utils.io import read_image
from hloc.visualization import plot_images

from megadepth.utils.projections import get_camera_poses
from megadepth.visualization.view_projections import create_view_projection_figure, pca

In [ ]:
base_dir = Path("/Volumes/Extreme_SSD/MegaDepth/scenes")

scene = "0015"

scene_dir = base_dir / scene
image_dir = scene_dir / "images"
sparse_dir = scene_dir / "sparse"
dense_dir = scene_dir / "dense"

for idx, name in enumerate(os.listdir(sparse_dir)):
    print(f"{idx}: {name}")

In [ ]:
# get number of models
model_name = os.listdir(sparse_dir)[1]

model_dir = sparse_dir / model_name / "hloc" / "models"

n_images = {}
for model_id in os.listdir(model_dir):
    if model_id.startswith("."):
        continue

    if model_id == "0":
        reconstruction = pycolmap.Reconstruction(str(model_dir.parent))
    else:
        reconstruction = pycolmap.Reconstruction(str(model_dir / model_id))
    n_images[model_id] = len(reconstruction.images)


fig, ax = plt.subplots()

ids = sorted(n_images.keys())
values = [n_images[id] for id in ids]

ax.bar(ids, values)
ax.set_xlabel("Model ID")
ax.set_ylabel("Number of images")
ax.set_title(f"Scene {scene} has {len(ids)} models")
plt.show()

In [ ]:
base_model = pycolmap.Reconstruction(str(model_dir.parent))

for model_id in sorted(n_images.keys()):
    if model_id.startswith("."):
        continue

    if model_id == "0":
        reconstruction = pycolmap.Reconstruction(str(model_dir.parent))
    else:
        reconstruction = pycolmap.Reconstruction(str(model_dir / model_id))

    # get camera poses and points
    camera_poses = get_camera_poses(reconstruction)
    points = np.array([p.xyz for p in reconstruction.points3D.values() if p.track.length() > 6])
    align = pca(camera_poses)

    # print summary
    print(f"{model_id} {reconstruction.summary()}")

    # plot camera poses and points
    create_view_projection_figure([align(points)], limit=3, s=0.1, alpha=.6)

    # plot sample images
    sample_images = np.random.choice([im.name for im in reconstruction.images.values()], 5)
    plot_images(
        [read_image(image_dir / fname) for fname in sample_images],
    )
    plt.show()